### Chat History OakLang Graph
Here we'll be interacting with a server that's exposing a chat bot with memmory history being persisted on the backend.

In [ ]:
import uuid
from langserve import RemoteRunnable

session_id = str(uuid.uuid4())
cheese = RemoteRunnable("http://localhost:8000/cheese")

### Quick Graph Test

In [ ]:
# define the thread_id
config = {"configurable": {"thread_id": "001"}}

In [ ]:
# Define the input and invoke
user_input = "Ey, qué tal?"
response = cheese.invoke({"messages": [("human", user_input)]}, config)
# to filter the message
print(response["messages"][-1].content)

In [ ]:
# Define the input and invoke
user_input = "Puedes resumirme los últimos 25 mensajes"
response = cheese.invoke({"messages": [("human", user_input)]}, config)
# to filter the message
print(response["messages"][-1].content)

### Live Chat Cheese-Chatter Graph

In [ ]:
## Live chat via stream
# define the thread_id
config = {"configurable": {"thread_id": "002"}}

while True:
    user_input = input("Human: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Cheese-Chatter: Adiós!")
        break
    else:
        print(f"Human: {user_input}") # You add this to see the user input
    for event in cheese.stream({"messages": [("human", user_input)]}, config): 
        for key, value in event.items():
            if key == 'cheeseagent' and len(value["messages"][-1].content) > 0:
                print("Cheese-Chatter:", value["messages"][-1].content)     

In [ ]:
## Follow the nodes via stream
# define the thread_id
config = {"configurable": {"thread_id": "002"}}

# define the input
user_input = "Do you know what is an atom?"

for output in cheese.stream({"messages": [("human", user_input)]}, config):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

### Request API

In [ ]:
import requests

In [ ]:
response = requests.post(
    url='http://localhost:8000/cheese/invoke',
    json={
        'input': {
            "messages": [("human", "Hola, cómo estás")],
        },
        'config': {
            'configurable': {
                "thread_id": "004"
            }
        }
    }
)

print(response.json()["output"]["messages"][-1])

In [ ]:
n_messages=25

url = f'http://192.168.1.224:8083/api/messages?size={n_messages}'
response = requests.get(url)

if response.status_code != 200:
    raise ValueError(f"Error: {n_messages} is not a valid argument")

data = response.json()

# Filter the id and timestand and reverse the messages
parsed_data = [{k: v for k, v in item.items() if k != 'timestamp' and k != 'id'} for item in reversed(data)]

parsed_data